In [2]:
import numpy as np
import os
import pandas as pd
import pyodbc
import time
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from collections import Counter
from datetime import datetime
import sys
import gc
from scipy.sparse import *
import torch
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle 
import random
import math
from joblib import dump, load
import matplotlib

from models import *
sys.path.append('../utils')
from dl_eval_utils import *
from eval_utils import *

# Choose best model based on hyperparameter tuning

In [ ]:
model_path = "MODEL PATH"
df_perf_lstm = pd.read_csv(model_path + 'MDCD_snomed_11_15_lstm_gridsearch.csv')
df_perf_trans = pd.read_csv(model_path + 'MDCD_snomed_11_15_transformer_gridsearch.csv')
df_perf_lstm.sort_values('AUPRC', ascending=False).head()

In [ ]:
df_perf_trans.sort_values('AUPRC', ascending=False).head(10)

In [5]:
print(df_perf_trans.iloc[109]['config'])
# transformer_dimFF128_hs256_nl2_lr0.0001_dropout0.3_wd0.001_weights1_EmbPE.pt


{'hidden_size': 256, 'dim_feedforward': 128, 'num_layers': 2, 'weights': tensor([1, 1], device='cuda:0'), 'emb_first': True, 'dropout': 0.3, 'learning_rate': 0.0001, 'weight_decay': 0.001}


# Get overall model performance

In [13]:
path = 'PATH'
raw_path = path + 'RAW DATA PATH'
int_path = path + 'INTERMEDIATE PATH'
result_data_path = path + 'RESULTS PATH'


df_pop = pd.read_csv(raw_path+'population.csv')

# X_mean = torch.load(int_path + 'MDCD_10_30_grud_means.pt')
with open(int_path + "MDCD_11_15_dl_colnames_snomed", "rb") as fp:   # Unpickling
    save_cols = pickle.load(fp)
    
# testing_clf = LSTMModel(input_size = len(save_cols), hidden_size=128, embedding_size = 1024, num_layers=2, dropout = 0.1, output_size=1)
testing_clf = TransformerModelEmbPE(hidden_size=256, dim_feedforward=128, num_layers=2, num_heads=4, dropout=0.3, n_features = len(save_cols))
# testing_clf = GRUD(len(save_cols), len(save_cols), len(save_cols), X_mean, output_last = True)
testing_clf.load_state_dict(torch.load(model_path + 'transformer_dimFF128_hs256_nl2_lr0.0001_dropout0.3_wd0.001_weights1_EmbPE.pt'))
testing_clf.eval()

device = torch.device("cuda")
testing_clf.to(device)

val_loader = torch.load(int_path + 'MDCD_11_15_dl_snomed_val_loader_unshuffled.pth')
test_loader = torch.load(int_path + 'MDCD_11_15_dl_snomed_test_loader_unshuffled.pth')

# Define metrics of interest
metric_functions = {
    'AUROC': roc_auc_score,
    'AUPRC': average_precision_score,
    'Sensitivity': sensitivity,
    'Specificity': specificity,
    'PPV': ppv_score}
binary_funcs = ['Sensitivity', 'Specificity', 'PPV']

/tmp/ipykernel_3727008/1049533622.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pop = pd.read_csv(raw_path+'population.csv')
/usr/lib/python3/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [14]:
# Get cutoff
val_modeloutput = ModelOutput(val_loader, testing_clf, 31)
y_valtrue, y_valpred = val_modeloutput.get_output_vals()
val_labels = pd.DataFrame(np.asarray([y_valtrue, y_valpred]).T, columns = ['sz_flag', 'pred_prob'])
cutoff_prob = val_modeloutput.get_cutoff_prob(y_valtrue, y_valpred, stored_data_path = model_path, save_filename = None)

Cutoff Probability: 0.05039993


In [15]:
# Get test performance
test_modeloutput = ModelOutput(test_loader, testing_clf, 31)
y_testtrue, y_testpred = test_modeloutput.get_output_vals()
test_labels = pd.DataFrame(np.asarray([y_testtrue, y_testpred]).T, columns = ['sz_flag', 'pred_prob'])

table2 = pd.DataFrame(columns = ['AUROC', 'AUPRC', 'Sensitivity', 'Specificity', 'PPV'])
table2 = create_table2_row(table2, val_labels, 'Validation', 'pred_prob', cutoff_prob, metric_functions, binary_funcs)
table2 = create_table2_row(table2, test_labels, 'Test', 'pred_prob', cutoff_prob, metric_functions, binary_funcs)


100%|████████████████████████████████████████████████████| 300/300 [00:22<00:00, 13.07it/s]


In [16]:
table2

,AUROC,AUPRC,Sensitivity,Specificity,PPV
Validation,"0.805 (0.786, 0.822)","0.339 (0.301, 0.376)","0.757 (0.723, 0.788)","0.725 (0.713, 0.735)","0.201 (0.184, 0.218)"
Test,"0.796 (0.78, 0.809)","0.312 (0.286, 0.338)","0.727 (0.697, 0.753)","0.714 (0.707, 0.721)","0.188 (0.176, 0.2)"
